In [23]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import csv
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

from sklearn import svm
import nltk
from nltk import bigrams
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud


In [ ]:
train_df = pd.read_csv("train.csv")
train_df.head()


In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
X = train_df.drop(["id"],axis=1)
X.head()


In [ ]:
key = X["keyword"].value_counts().index[0]
print(key)
loc = X["location"].value_counts().index[0]
print(loc)


In [ ]:
train_df['keyword'] = train_df['keyword'].fillna(train_df['keyword'].value_counts().idxmax())
train_df['location'] = train_df['location'].fillna(train_df['location'].value_counts().idxmax())
train_df.isnull().sum()


In [ ]:
X_new = train_df.drop(["target"],axis=1)
X_new.head()


In [ ]:
y = train_df["target"]
y.shape


In [ ]:
test_df.isnull().sum()

In [ ]:
test_df['keyword'] = test_df['keyword'].fillna(test_df['keyword'].value_counts().idxmax())
test_df['location'] = test_df['location'].fillna(test_df['location'].value_counts().idxmax())
test_df.isnull().sum()

In [ ]:
X_new = X_new.drop(["id"],axis=1)
X_new.head()

In [ ]:
X_new.replace("[^a-zA-Z]", " ",regex = True, inplace = True)
X_new.head()


In [ ]:
for i in X_new.columns:
    X_new[i] = X_new[i].str.lower()
X_new.head()


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


In [ ]:
X_new['keyword'].apply(lambda x: [item for item in x if item not in stop])
X_new['location'].apply(lambda x: [item for item in x if item not in stop])
X_new['text'].apply(lambda x: [item for item in x if item not in stop])
print(X_new.shape)


In [ ]:
ori_test = test_df.drop(["id"],axis=1)

ori_test.replace("[^a-zA-Z]", " ",regex = True, inplace = True)
ori_test.head()


In [ ]:
for i in ori_test.columns:
    ori_test[i] = ori_test[i].str.lower()
ori_test.head()

In [ ]:
ori_test['keyword'].apply(lambda x: [item for item in x if item not in stop])
ori_test['location'].apply(lambda x: [item for item in x if item not in stop])
ori_test['text'].apply(lambda x: [item for item in x if item not in stop])
print(ori_test.shape)

In [ ]:
X_new["sentence"] = X_new['keyword'] + " " + X_new['text']
train_text = np.array(X_new["sentence"])
print(train_text[0])

In [ ]:
# test dataset
ori_test["sentence"] = ori_test['keyword'] + " " + ori_test['text']
test_text = np.array(ori_test["sentence"])
print(test_text[0])

1. Using Bag-of-Words

In [ ]:
from sklearn.model_selection import train_test_split
x_trainb, x_testb, y_trainb, y_testb = train_test_split(train_text,y,test_size=0.25,random_state=0)
print(x_trainb.shape)
print(x_testb.shape)
print(y_trainb.shape)
print(y_testb.shape)

In [ ]:
bow = CountVectorizer(max_features=6000, ngram_range=(1,3))
bow_df = bow.fit_transform(x_trainb)
bow_df.shape

In [ ]:
bow_X = bow_df
print(bow_X.shape)

In [ ]:
keyword = bow.get_feature_names()
X_trainb = bow.transform(x_trainb)
X_testb = bow.transform(x_testb)
X_testb_new = bow.transform(test_text)

1.1 Random Forest Classifier

In [ ]:
rc = RandomForestClassifier(max_depth = 400 ,random_state=0, n_estimators = 300)
rc.fit(X_trainb,y_trainb) 

In [46]:
predb1 = rc.predict(X_testb)
matrix = confusion_matrix(y_testb,predb1)
print(matrix)
score = accuracy_score(y_testb,predb1)
print(score)
report = classification_report(y_testb,predb1)
print(report)
predictionb1 = rc.predict(X_testb_new)
predictionb1

[[1017   90]
 [ 316  481]]
0.7867647058823529
              precision    recall  f1-score   support

           0       0.76      0.92      0.83      1107
           1       0.84      0.60      0.70       797

    accuracy                           0.79      1904
   macro avg       0.80      0.76      0.77      1904
weighted avg       0.80      0.79      0.78      1904



array([0, 0, 0, ..., 1, 1, 0])

1.2 Logistic Regression

In [47]:
LR = LogisticRegression().fit(X_trainb,y_trainb)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [48]:
predb2 = LR.predict(X_testb)
matrix = confusion_matrix(y_testb,predb2)
print(matrix)
score = accuracy_score(y_testb,predb2)
print(score)
report = classification_report(y_testb,predb2)
print(report)
predictionb2 = LR.predict(X_testb_new)
predictionb2

[[961 146]
 [216 581]]
0.8098739495798319
              precision    recall  f1-score   support

           0       0.82      0.87      0.84      1107
           1       0.80      0.73      0.76       797

    accuracy                           0.81      1904
   macro avg       0.81      0.80      0.80      1904
weighted avg       0.81      0.81      0.81      1904



array([1, 0, 1, ..., 1, 1, 1])

1.3 SVM

In [49]:
SVM = svm.SVC().fit(X_trainb, y_trainb)

In [50]:
predb3 = SVM.predict(X_testb)
matrix = confusion_matrix(y_testb,predb3)
print(matrix)
score = accuracy_score(y_testb,predb3)
print(score)
report = classification_report(y_testb,predb3)
print(report)
predictionb3 = SVM.predict(X_testb_new)
predictionb3

[[1018   89]
 [ 277  520]]
0.8077731092436975
              precision    recall  f1-score   support

           0       0.79      0.92      0.85      1107
           1       0.85      0.65      0.74       797

    accuracy                           0.81      1904
   macro avg       0.82      0.79      0.79      1904
weighted avg       0.81      0.81      0.80      1904



array([0, 0, 1, ..., 1, 1, 0])

2. Using Tf-Idf Vectorizer

In [ ]:
from sklearn.model_selection import train_test_split
x_traint, x_testt, y_traint, y_testt = train_test_split(train_text,y,test_size=0.25,random_state=0)
print(x_traint.shape)
print(x_testt.shape)
print(y_traint.shape)
print(y_testt.shape)

In [ ]:
tfidf = TfidfVectorizer(max_features=6000, ngram_range=(1,3))
tfidf_X = tfidf.fit_transform(x_traint)
tfidf_X.shape

In [ ]:
keyword = tfidf.get_feature_names()
X_traint = tfidf.transform(x_traint)
X_testt = tfidf.transform(x_testt)
X_testt_new = tfidf.transform(test_text)

2.1 Random Forest Classifier

In [54]:
rc = RandomForestClassifier(max_depth = 400 ,random_state=0, n_estimators = 300)
rc.fit(X_traint,y_traint) 

RandomForestClassifier(max_depth=400, n_estimators=300, random_state=0)

In [55]:
predt1 = rc.predict(X_testt)
matrix = confusion_matrix(y_testt,predt1)
print(matrix)
score = accuracy_score(y_testt,predt1)
print(score)
report = classification_report(y_testt,predt1)
print(report)
predictiont1 = rc.predict(X_testt_new)
predictiont1

[[1020   87]
 [ 318  479]]
0.7872899159663865
              precision    recall  f1-score   support

           0       0.76      0.92      0.83      1107
           1       0.85      0.60      0.70       797

    accuracy                           0.79      1904
   macro avg       0.80      0.76      0.77      1904
weighted avg       0.80      0.79      0.78      1904



array([0, 0, 1, ..., 1, 1, 0])

2.2 Logistic Regression

In [56]:
LR = LogisticRegression().fit(X_traint,y_traint)

In [57]:
predt2 = LR.predict(X_testt)
matrix = confusion_matrix(y_testt,predt2)
print(matrix)
score = accuracy_score(y_testt,predt2)
print(score)
report = classification_report(y_testt,predt2)
print(report)
predictiont2 = LR.predict(X_testt_new)
predictiont2

[[980 127]
 [237 560]]
0.8088235294117647
              precision    recall  f1-score   support

           0       0.81      0.89      0.84      1107
           1       0.82      0.70      0.75       797

    accuracy                           0.81      1904
   macro avg       0.81      0.79      0.80      1904
weighted avg       0.81      0.81      0.81      1904



array([1, 0, 1, ..., 1, 1, 1])

2.3 SVM

In [58]:
SVM = svm.SVC().fit(X_traint, y_traint)

In [59]:
predt3 = SVM.predict(X_testt)
matrix = confusion_matrix(y_testt,predt3)
print(matrix)
score = accuracy_score(y_testt,predt3)
print(score)
report = classification_report(y_testt,predt3)
print(report)
predictiont3 = SVM.predict(X_testt_new)
predictiont3

[[1002  105]
 [ 262  535]]
0.8072478991596639
              precision    recall  f1-score   support

           0       0.79      0.91      0.85      1107
           1       0.84      0.67      0.74       797

    accuracy                           0.81      1904
   macro avg       0.81      0.79      0.79      1904
weighted avg       0.81      0.81      0.80      1904



array([1, 0, 1, ..., 1, 1, 1])

3. Using Hashing Vectorizer

In [60]:
from sklearn.model_selection import train_test_split
x_trainh, x_testh, y_trainh, y_testh = train_test_split(train_text,y,test_size=0.25,random_state=0)
print(x_trainh.shape)
print(x_testh.shape)
print(y_trainh.shape)
print(y_testh.shape)

(5709,)
(1904,)
(5709,)
(1904,)


In [61]:
hv = HashingVectorizer(alternate_sign=False,n_features=6000, ngram_range=(1,3))
hv_X = hv.fit_transform(x_trainh)
hv_X.shape

(5709, 6000)

In [63]:
keyword = hv.get_params()
X_trainh = hv.transform(x_trainh)
X_testh = hv.transform(x_testh)
X_testh_new = hv.transform(test_text)

3.1 Random Forest Classifier

In [64]:
rc = RandomForestClassifier(max_depth = 400 ,random_state=0, n_estimators = 300)
rc.fit(X_trainh,y_trainh) 

RandomForestClassifier(max_depth=400, n_estimators=300, random_state=0)

In [65]:
predh1 = rc.predict(X_testh)
matrix = confusion_matrix(y_testh,predh1)
print(matrix)
score = accuracy_score(y_testh,predh1)
print(score)
report = classification_report(y_testh,predh1)
print(report)
predictionh1 = rc.predict(X_testh_new)
predictionh1

[[976 131]
 [311 486]]
0.7678571428571429
              precision    recall  f1-score   support

           0       0.76      0.88      0.82      1107
           1       0.79      0.61      0.69       797

    accuracy                           0.77      1904
   macro avg       0.77      0.75      0.75      1904
weighted avg       0.77      0.77      0.76      1904



array([0, 0, 0, ..., 1, 1, 0])

3.2 Logistic Regression

In [66]:
LR = LogisticRegression().fit(X_trainh,y_trainh)

In [67]:
predh2 = LR.predict(X_testh)
matrix = confusion_matrix(y_testh,predh2)
print(matrix)
score = accuracy_score(y_testh,predh2)
print(score)
report = classification_report(y_testh,predh2)
print(report)
predictionh2 = LR.predict(X_testh_new)
predictionh2

[[974 133]
 [304 493]]
0.770483193277311
              precision    recall  f1-score   support

           0       0.76      0.88      0.82      1107
           1       0.79      0.62      0.69       797

    accuracy                           0.77      1904
   macro avg       0.77      0.75      0.75      1904
weighted avg       0.77      0.77      0.76      1904



array([0, 0, 0, ..., 1, 1, 0])

3.3 SVM

In [68]:
SVM = svm.SVC().fit(X_trainh, y_trainh)

In [69]:
predh3 = SVM.predict(X_testh)
matrix = confusion_matrix(y_testh,predh3)
print(matrix)
score = accuracy_score(y_testh,predh3)
print(score)
report = classification_report(y_testh,predh3)
print(report)
predictionh3 = SVM.predict(X_testh_new)
predictionh3

[[1007  100]
 [ 319  478]]
0.779936974789916
              precision    recall  f1-score   support

           0       0.76      0.91      0.83      1107
           1       0.83      0.60      0.70       797

    accuracy                           0.78      1904
   macro avg       0.79      0.75      0.76      1904
weighted avg       0.79      0.78      0.77      1904



array([0, 0, 0, ..., 1, 1, 0])

In [70]:
data = {'id':test_df["id"],'target':predictionb3}
output = pd.DataFrame(data, columns = ['id','target'])
output.index = test_df.index

In [71]:
output.to_csv("submission.csv", index = False)    


In [72]:
a = pd.read_csv("submission.csv")
a

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1
